# Inference and Reasoning with Bayesian Networks

In this assignment we were interested in predicting the outcome on an election, based on a survey of voters who gave their preferred candidate, which region they live in, their gender, political leaning, employment status and age. These features have strong dependencies with one another, so a classification model which fails to take these into account (e.g. Naive Bayes) would be inappropriate. We have used Bayesian Networks to model the relationships between the features and makes inferences accordingly. 

Please evaluate this cell to enable LaTeX macros. 
$\newcommand{\dotprod}[2]{\left\langle #1, #2 \right\rangle}$
$\newcommand{\onevec}{\mathbb{1}}$

In [1]:
import itertools, copy
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Image

%matplotlib inline

##  Graphical Models

### Problem setting

We are interested to predict the outcome of the election in an imaginary country, called Under Some Assumptions (USA). There are four candidates for whom the citizens can **Vote** for: Bernie, Donald, Hillary, and Ted. The citizens live in four **Region**s: north, south, east and west. We have general demographic information about the people, namely: **Gender** (male, female) and **Hand**edness (right, left). Based on surveys done by an external company, we believe that the **Region** and **Gender** affects whether the people use their **Jacket**s full time, part time or never. Surprisingly, the company told us that the **Age** of their shoes (new, worn, old) depends on how often they wear their **Jacket**s. Furthermore, the **Gender** and their preferred **Hand** affects the **Colour** of their hat (white, black). Finally, surveys say that the citizens will **Vote** based on their **Region**, **Age** of their shoes and **Colour** of their hats.

The directed graphical model is depicted below:

In [2]:
Image(url="https://machlearn.gitlab.io/isml2017/assignments/election_model.png")

### Conditional probability tables

After paying the survey firm some more money, they provided the following conditional probability tables.

|$p(R)$ | R=n | R=s | R=e | R=w |
|:-----:|:--:|:--:|:--:|:--:|
|marginal| 0.2 | 0.1 | 0.5 | 0.2 |

|$p(G)$ | G=m | G=f |
|:-----:|:--:|:--:|
|marginal| 0.3 | 0.7 |

|$p(H)$ | H=r | H=l |
|:-----:|:--:|:--:|
|marginal| 0.9 | 0.1 |

| $p(J|R,G)$ | R=n,G=m | R=n,G=f | R=s,G=m | R=s,G=f | R=e,G=m | R=e,G=f | R=w,G=m | R=w,G=f |
|:-----:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|**J**=full $\quad$  |0.9 |0.8 |0.1 | 0.3 |0.4 |0.01| 0.02 | 0.2  |
|**J**=part $\quad$  |0.08|0.17|0.03| 0.35|0.05|0.01| 0.2  | 0.08 |
|**J**=never $\quad$ |0.02|0.03|0.87| 0.35|0.55|0.98| 0.78 | 0.72 |

| $p(A|J)$ | J=full | J=part | J=never |
|:-----:|:--:|:--:|:--:|
|**A**=new  |0.01|0.96|0.3|
|**A**=worn |0.98|0.03|0.5|
|**A**=old  |0.01|0.01|0.2|

| $p(C|G,H)$ | G=m,H=r | G=m,H=l | G=f,H=r | G=f,H=l |
|:-----:|:--:|:--:|:--:|:--:|
|**C**=black $\quad$ |0.9 |0.83 |0.17 | 0.3 |
|**C**=white $\quad$ |0.1 |0.17|0.83 | 0.7 |

The final conditional probability table is given by the matrix below. The order of the rows and columns are also given below.

In [3]:
vote_column_names = ['north,new,black', 'north,new,white', 'north,worn,black', 'north,worn,white', 
                'north,old,black', 'north,old,white', 'south,new,black', 'south,new,white', 
                'south,worn,black', 'south,worn,white', 'south,old,black', 'south,old,white', 
                'east,new,black', 'east,new,white', 'east,worn,black', 'east,worn,white', 
                'east,old,black', 'east,old,white', 'west,new,black', 'west,new,white', 
                'west,worn,black', 'west,worn,white', 'west,old,black', 'west,old,white']

vote_outcomes = ('bernie','donald','hillary','ted')

vote_pmf_array = np.array(
        [
            [0.1,0.1,0.4,0.02,0.2,0.1,0.1,0.04,0.2,0.1,0.1 ,0.1,0.4 ,0.1 ,0.1,0.1 ,0.1,0.04,0.3,0.2,0.1,0.3,0.34,0.35],
            [0.3,0.4,0.2,0.5 ,0.1,0.2,0.1,0.5 ,0.1,0.2,0.5 ,0.3,0.2 ,0.42,0.2,0.67,0.4,0.4 ,0.1,0.1,0.5,0.1,0.1 ,0.1],
            [0.5,0.4,0.3,0.3 ,0.5,0.6,0.6,0.3 ,0.5,0.4,0.36,0.3,0.28,0.3 ,0.4,0.1 ,0.4,0.16,0.4,0.2,0.3,0.3,0.4 ,0.5],
            [0.1,0.1,0.1,0.18,0.2,0.1,0.2,0.16,0.2,0.3,0.04,0.3,0.12,0.18,0.3,0.13,0.1,0.4 ,0.2,0.5,0.1,0.3,0.16,0.05],
        ]
)

The 7 conditional probability tables in are encoded in python below. 

**Base your subsequent computations on these objects**.

In [4]:
class RandomVariable(object):
    def __init__(self, name, parents, outcomes, pmf_array):
        assert isinstance(name, str)
        assert all(isinstance(_, RandomVariable) for _ in parents)
        assert isinstance(outcomes, tuple)
        assert all(isinstance(_, str) for _ in outcomes)
        assert isinstance(parents, tuple)
        assert isinstance(pmf_array, np.ndarray)
        keys = tuple(map(tuple, itertools.product(*[_.outcomes for _ in parents])))
        assert np.allclose(np.sum(pmf_array, 0), 1)
        expected_shape = (len(outcomes), len(keys))
        assert pmf_array.shape == expected_shape, (pmf_array.shape, expected_shape)
        pmfs = {k: {outcome: probability for outcome, probability in zip(outcomes, probabilities)} 
                for k, probabilities in zip(keys, pmf_array.T)}
        self.name, self.parents, self.outcomes, self.pmfs = name, parents, outcomes, pmfs


class BayesianNetwork(object):
    def __init__(self, *random_variables):
        assert all(isinstance(_, RandomVariable) for _ in random_variables)
        self.random_variables = random_variables
        

region_pmf_array = np.array([[0.2, 0.1, 0.5, 0.2]]).T
region = RandomVariable(
    name='region',
    parents=tuple(),
    outcomes=('north', 'south', 'east', 'west'), 
    pmf_array = region_pmf_array,
)

gender_pmf_array = np.array([[0.3, 0.7]]).T
gender = RandomVariable(
    name='gender',
    parents=tuple(),
    outcomes=('male', 'female'), 
    pmf_array = gender_pmf_array
)

hand_pmf_array = np.array([[0.9, 0.1]]).T
hand = RandomVariable(
    name='hand',
    parents=tuple(),
    outcomes=('left', 'right'), 
    pmf_array = hand_pmf_array
)

jacket_pmf_array = np.array(
        [
            [0.9,0.8,0.1,0.3,0.4,0.01,0.02,0.2],
            [0.08,0.17,0.03,0.35,0.05,0.01,0.2,0.08],
            [0.02,0.03,0.87,0.35,0.55,0.98,0.78,0.72],
        ]
    )
jacket = RandomVariable(
    name='jacket',
    parents=(region, gender),
    outcomes=('full', 'part', 'never'), 
    pmf_array = jacket_pmf_array
)

age_pmf_array = np.array(
        [
            [0.01,0.96,0.3],
            [0.98,0.03,0.5],
            [0.01,0.01,0.2],
        ]
    )
age = RandomVariable(
    name='age',
    parents=(jacket, ),
    outcomes=('new', 'worn', 'old'), 
    pmf_array = age_pmf_array
)

colour_pmf_array = np.array(
        [
            [0.9,0.83,0.17,0.3],
            [0.1,0.17,0.83,0.7],
        ]
    )
colour = RandomVariable(
    name='colour',
    parents=(gender, hand),
    outcomes=('black', 'white'),
    pmf_array = colour_pmf_array
)

vote = RandomVariable(
    name='vote',
    parents=(region, age, colour),
    outcomes=vote_outcomes,
    pmf_array = vote_pmf_array
)


election_model = BayesianNetwork(region, gender, hand, jacket, age, colour, vote)

### 1A Joint distribution of a subset

- Compute the joint distribution of **Jacket**, **Region** and **Gender**. Print in a tab separated format the resulting distribution.

In [5]:
def print_joint_JRG():
    '''
    Prints the joint distribution of Jacket, Region, Gender.
    Uses the fact that Region and Gender are independent, so
    p(J, R, G) = p(J | R, G) * p(R) * p(G)
    '''
    for jacket_outcome in jacket.outcomes:
        print('jacket region gender joint_pmf')
        print('------------------------------')
        for region_outcome in region.outcomes:
            for gender_outcome in gender.outcomes:
                JRG_joint = jacket.pmfs[(region_outcome, gender_outcome)][jacket_outcome] * \
                region.pmfs[()][region_outcome] * gender.pmfs[()][gender_outcome]
                print(jacket_outcome + ' '*(7-len(jacket_outcome)) + region_outcome + ' '*(7-len(region_outcome))\
                      + gender_outcome + ' '* (7 - len(gender_outcome)) + str(JRG_joint) )
        print(' ')
        
print_joint_JRG()

jacket region gender joint_pmf
------------------------------
full   north  male   0.054
full   north  female 0.112
full   south  male   0.003
full   south  female 0.021
full   east   male   0.06
full   east   female 0.0035
full   west   male   0.0012
full   west   female 0.028
 
jacket region gender joint_pmf
------------------------------
part   north  male   0.0048
part   north  female 0.0238
part   south  male   0.0009
part   south  female 0.0245
part   east   male   0.0075
part   east   female 0.0035
part   west   male   0.012
part   west   female 0.0112
 
jacket region gender joint_pmf
------------------------------
never  north  male   0.0012
never  north  female 0.0042
never  south  male   0.0261
never  south  female 0.0245
never  east   male   0.0825
never  east   female 0.343
never  west   male   0.0468
never  west   female 0.1008
 


### 1B1 Variable Ordering

1. Implement a function which determines an appropriate ordering of the variables for use in the following scheme:
    - For the first node R, draw a sample from p(R).
    - For each subsequent node, draw a sample from the conditional distribution $p(X \,|\, \text{parents}(X))$ where $\text{parents}(X)$ are the parents of the variable $X$ in the graphical model.
- Use your function to compute such an ordering and print the result in a human friendly format.


1.

In [6]:
def child_dictionary(network):
    '''
    For a network of random variables, this function returns a dictionary whose keys are 
    the random variables and the values are dictionaries containing a list of children 
    of the key.
    
    This function makes it easier to implement a depth first search of the network.
    '''
    child_dict = { random_variable : [] for random_variable in network.random_variables }
    
    for random_variable in network.random_variables:
        for parent in random_variable.parents:
            child_dict[parent].append(random_variable)
    return child_dict

def dfs(network):
    '''
    Takes in a network and returns a list of nodes ordered so that any entries
    parents are earlier in the list (a topological ordering). Uses Depth First Search.
    If not a DAG, will return "Not a DAG".
    '''
    mark_dict = {random_variable : 'unmarked' for random_variable in network.random_variables}
    child_dict = child_dictionary(network)
    
    L = [] # List that eventually stores nodes in correct order
    def visit(node):
        if mark_dict[node] == 'temp_mark':
            return "Not a DAG"
        if mark_dict[node] == 'unmarked':
            mark_dict[node] = 'temp_mark'
            for child in child_dictionary(network)[node]:
                visit(child)
            mark_dict[node] = 'marked'
            L.insert(0,node)
            
    def unmarked(mDic):
        return [key for key, value in mark_dict.items() if value == 'unmarked']
    
    while unmarked(mark_dict):
        visit(unmarked(mark_dict)[0])
    return L
    

2.

In [7]:
ordering = dfs(election_model)

for i, node in enumerate(ordering):
    print(i+1, node.name)

1 hand
2 gender
3 colour
4 region
5 jacket
6 age
7 vote


### 1B2 Sampling

1. Given the ordering you have determined above, implement the sampler described in the previous question. If you were unable to compute the ordering in the previous question, use ``ordering = (hand, region, gender, colour, jacket, age, vote)``.
2. Draw a single sample and print the result in a human friendly format.


1.

In [8]:
def sample_node(node, parents_values):
    '''
    Given a node and the values of its parents, function returns a sample outcome
    of the node conditional on the values of it's parents. Uses numpy's random.multinomial
    to sample over the various options with their specified probabilities.
    '''
    outcome_list = [outcome for outcome in sorted(node.pmfs[parents_values].keys())]
    outcome_probs = [ node.pmfs[parents_values][outcome] for outcome in sorted(node.pmfs[parents_values].keys())]
    
    return outcome_list[np.where(np.random.multinomial(1, outcome_probs) ==1)[0][0]]
    

def sample_network(network):
    '''
    Given a DAG, function returns a sample from the network by first computing
    a topological order, then sampling each node conditional on the values of it's parents.
    '''
    ordered_nodes = dfs(network)
    sample_dict = dict()

    for node in ordered_nodes:
        parent_values = tuple( sample_dict[parent.name] for parent in node.parents)
        sample_dict[node.name] = sample_node(node, parent_values)
    return sample_dict
            
                

2.

In [9]:
sample = sample_network(election_model)

print('Variable  Outcome')
print('----------------')
for node, outcome in sample.items():
    print( node + ' '*(10 - len(node))  + outcome)

Variable  Outcome
----------------
hand      left
gender    male
colour    black
region    east
jacket    part
age       new
vote      donald


### 1B3 Marginals

1. Calculate (and show in LaTeX) the marginal distribution for **Jacket**.
- Implement a function which computes the marginal distribution of each variable. It should make use of the ordering used by your sampler.
- Implement a function which takes a list of samples the model and computes the empirical marginal distribution of each variable.
- Plot the theoretical and approximate marginals which you obtain along with the absolute percent error between the two, in the format below:

1.By summing the right hand columns of each block printing in Question 1A, we can see that the marginal distribution for Jacket is given by: 
$$ \mathbb{P}( \text{Jacket} = \text{'full'} ) = 0.2827 \\
\mathbb{P}( \text{Jacket} = \text{'part'} ) = 0.0882 \\
\mathbb{P}( \text{Jacket} = \text{'never'} ) = 0.6291 \\
$$

2.

In [10]:
def joint_pmf(outcome_dict, network):
    '''
    Takes a network and a dictionary whose keys are all node names of network and values are their outcomes.
    Returns the joint probability of that set of outcomes. We use the factorisation implied by the graphical
    structure.
    '''
    prod = 1
    for node in dfs(network):
        prod *= node.pmfs[ tuple(outcome_dict[parent] for parent in node.parents) ][outcome_dict[node]]
    return prod

def marginals(network):
    '''
    Given a network, this function returns a dictionary whose keys are the nodes of the network
    and values which are dictionaries describing that nodes marginal distribution. It does this
    by summing the joint_pmf over the variables to be marginalized. 
    '''
    marginal_dict = dict()
    
    for remaining_node in dfs(network):
        
        marginalized_nodes = dfs(network)
        marginalized_nodes.remove(remaining_node)
        node_marginal_dict = dict()
        for remaining_node_outcome in remaining_node.outcomes:
            
            remaining_node_cumsum = 0
            for outcome in itertools.product(*tuple(node.outcomes for node in marginalized_nodes)):
                
                outcome_dictionary = dict(zip(marginalized_nodes, outcome))
                outcome_dictionary[remaining_node] = remaining_node_outcome
                remaining_node_cumsum += joint_pmf(outcome_dictionary, network)
            node_marginal_dict[remaining_node_outcome] = remaining_node_cumsum

        marginal_dict[remaining_node.name] = node_marginal_dict
    return marginal_dict

3.

In [11]:
def sampled_marginals(network, number_of_samples):
    '''
    Given a network and a number of samples, the function returns a dictionary whose keys
    are the nodes of the network and whose values approximate that nodes marginal distribution.
    The values for the marginals are derived from sampling the network using the previous sampler
    function.
    '''
    marginals_dict = dict()
    sample_list = [ sample_network(election_model) for i in range(number_of_samples)]
    
    for node in network.random_variables:
        node_samples = np.array( [sample[node.name] for sample in sample_list])
        unique, counts = np.unique(node_samples, return_counts=True)
        relative_frequencies = counts / number_of_samples
        node_marginal_dict = dict(zip(unique, relative_frequencies))
        marginals_dict[node.name] = node_marginal_dict
        
    return marginals_dict

4.

In [12]:
def print_marginals_exact_vs_sampled(network, number_of_samples):
    '''
    Prints a table comparing the exact and sampled marginal values for the nodes
    in network.
    '''
    header = 'outcome exact  approx error (%) '
    exact_marginals = marginals(network)
    approx_marginals = sampled_marginals(network, number_of_samples)
    
    for node in dfs(network):
        print(node.name)
        print(header)
        for outcome in node.outcomes:
            exact = exact_marginals[node.name][outcome]
            approx = approx_marginals[node.name][outcome]
            error = abs(approx - exact)/exact * 100
            print(outcome + ' '*(8-len(outcome)) + '%.2f   %.2f   %.2f' % (exact, approx, error) )
        print('')

print_marginals_exact_vs_sampled(election_model, 10**4)

hand
outcome exact  approx error (%) 
left    0.90   0.90   0.02
right   0.10   0.10   0.20

gender
outcome exact  approx error (%) 
male    0.30   0.31   4.53
female  0.70   0.69   1.94

colour
outcome exact  approx error (%) 
black   0.40   0.40   2.07
white   0.60   0.60   1.36

region
outcome exact  approx error (%) 
north   0.20   0.19   2.85
south   0.10   0.10   3.50
east    0.50   0.50   0.56
west    0.20   0.20   2.50

jacket
outcome exact  approx error (%) 
full    0.28   0.28   1.49
part    0.09   0.09   2.04
never   0.63   0.64   0.95

age
outcome exact  approx error (%) 
new     0.28   0.28   0.53
worn    0.59   0.59   0.12
old     0.13   0.13   0.56

vote
outcome exact  approx error (%) 
bernie  0.15   0.15   1.76
donald  0.35   0.34   1.48
hillary 0.30   0.30   2.91
ted     0.20   0.20   0.37



### 1B4 Easy conditional probabilities

Compute $p(X \,|\, G=\text{female})$ for all $X$ other than $G$,
1. Approximately, using your sampler
- Exactly, using your marginal calculating function from the previous question. Hint: what happens if you set  $p(G=\text{female})=1$ in your model?
- Plot the results side by side in the same format as the previous question.
- State for which variables other than $G$ the theoretical scheme above can be used to compute such conditionals, and why.

1.

In [13]:
def marginals_given_female_sampler(number_of_samples):
    '''
    Takens integer input and samples the network that many times.
    It then filters the samples for when G = female to approxinmate
    p(X | G = female) for all X other than G, returning the results
    in a dictionary.
    '''
    sample_list = [sample_network(election_model) for i in range(number_of_samples)]
    female_samples = list(filter(lambda sample: sample['gender']=='female', sample_list))
    node_list = dfs(election_model)
    node_list.remove(gender)
    node_dict = dict()
    for node in node_list:
        node_outcome_counter = { outcome: 0 for outcome in node.outcomes}

        for sample in female_samples:
            node_outcome_counter[sample[node.name]] += 1/len(female_samples)
        
        node_dict[node.name] = node_outcome_counter
    return node_dict

sample_marginals_given_female= marginals_given_female_sampler(10**4)
sample_marginals_given_female

{'age': {'new': 0.28708202343527217,
  'old': 0.13903972563589367,
  'worn': 0.573878250928827},
 'colour': {'black': 0.18505287224921402, 'white': 0.8149471277506943},
 'hand': {'left': 0.8981137467846758, 'right': 0.10188625321520246},
 'jacket': {'full': 0.2394969991426148,
  'never': 0.672620748785324,
  'part': 0.08788225207201937},
 'region': {'east': 0.4965704486996437,
  'north': 0.20834524149757183,
  'south': 0.09531294655615734,
  'west': 0.19977136324664257},
 'vote': {'bernie': 0.13432409259788258,
  'donald': 0.3876821949128421,
  'hillary': 0.26850528722492545,
  'ted': 0.2094884252643624}}

2.

In [14]:
_region = RandomVariable(
    name='region',
    parents=tuple(),
    outcomes=('north', 'south', 'east', 'west'), 
    pmf_array = region_pmf_array)

_gender_pmf_array = np.array([[0.0, 1.0]]).T
_gender = RandomVariable(
    name='gender',
    parents=tuple(),
    outcomes=('male', 'female'), 
    pmf_array = _gender_pmf_array)

_hand = RandomVariable(
    name='hand',
    parents=tuple(),
    outcomes=('left', 'right'), 
    pmf_array = hand_pmf_array)

_jacket = RandomVariable(
    name='jacket',
    parents=(_region, _gender),
    outcomes=('full', 'part', 'never'), 
    pmf_array = jacket_pmf_array)

_age = RandomVariable(
    name='age',
    parents=(_jacket, ),
    outcomes=('new', 'worn', 'old'), 
    pmf_array = age_pmf_array)

_colour = RandomVariable(
    name='colour',
    parents=(_gender, _hand),
    outcomes=('black', 'white'),
    pmf_array = colour_pmf_array)

_vote = RandomVariable(
    name='vote',
    parents=(_region, _age, _colour),
    outcomes=vote_outcomes,
    pmf_array = vote_pmf_array)

modified_election_model = BayesianNetwork(_region, _gender, _hand, _jacket, _age, _colour, _vote)

In [15]:
exact_marginals_given_female = marginals(modified_election_model)
del exact_marginals_given_female['gender']

exact_marginals_given_female

{'age': {'new': 0.29125000000000029,
  'old': 0.13825000000000004,
  'worn': 0.57050000000000056},
 'colour': {'black': 0.18299999999999991, 'white': 0.81700000000000028},
 'hand': {'left': 0.90000000000000013, 'right': 0.09999999999999995},
 'jacket': {'full': 0.23499999999999979,
  'never': 0.67500000000000004,
  'part': 0.089999999999999983},
 'region': {'east': 0.5,
  'north': 0.20000000000000012,
  'south': 0.099999999999999978,
  'west': 0.19999999999999998},
 'vote': {'bernie': 0.13311373599999998,
  'donald': 0.38725060750000045,
  'hillary': 0.26867528599999996,
  'ted': 0.21096037050000008}}

3.

In [16]:
header = 'outcome exact  approx error (%) '
exact_marginals = exact_marginals_given_female
approx_marginals = sample_marginals_given_female
node_list = dfs(election_model)
node_list.remove(gender)
for node in node_list:
    print(node.name)
    print(header)
    for outcome in node.outcomes:
        exact = exact_marginals[node.name][outcome]
        approx = approx_marginals[node.name][outcome]
        error = abs(approx - exact)/exact * 100
        print(outcome + ' '*(8-len(outcome)) + '%.2f   %.2f   %.2f' % (exact, approx, error) )
    print('')

hand
outcome exact  approx error (%) 
left    0.90   0.90   0.21
right   0.10   0.10   1.89

colour
outcome exact  approx error (%) 
black   0.18   0.19   1.12
white   0.82   0.81   0.25

region
outcome exact  approx error (%) 
north   0.20   0.21   4.17
south   0.10   0.10   4.69
east    0.50   0.50   0.69
west    0.20   0.20   0.11

jacket
outcome exact  approx error (%) 
full    0.23   0.24   1.91
part    0.09   0.09   2.35
never   0.68   0.67   0.35

age
outcome exact  approx error (%) 
new     0.29   0.29   1.43
worn    0.57   0.57   0.59
old     0.14   0.14   0.57

vote
outcome exact  approx error (%) 
bernie  0.13   0.13   0.91
donald  0.39   0.39   0.11
hillary 0.27   0.27   0.06
ted     0.21   0.21   0.70



4.

This works for any node without a parent, so $R, G, H.$ This is because:

\begin{align*} 
p( X \ | G =\text{female}) &= \frac{1}{p(G =\text{female})} \ \sum_{\text{all but  G}} p(X, G=\text{female}) \\
                           &= \frac{1}{p(G =\text{female})} \ \sum_{\text{all but  G}} p(H)\ p(G =\text{female})\ p(C \ | \ H, G =\text{female}) \ p(R) \ p(J \ | \ R, G =\text{female}) \ p(A \ | \ J) \ p(V \ | \ A, R, C) \\
                           &= \sum_{\text{all but  G}} p(H)\ p(C \ | \ H, G =\text{female}) \ p(R) \ p(J \ | \ R, G =\text{female}) \ p(A \ | \ J) \ p(V \ | \ A, R, C)
\end{align*}

and this matches the way we compute marginals in a modified model where $p(G = \text{female}) = 1.$ A similar calculation holds for $R, H.$

### 1B5 General conditional probabilities

1. Write down the expression of the joint probability $p(R,G,H,J,A,C,V)$ in terms of the conditional probabilities in the graphical model.
- Derive $p(G = male \,|\, V = Donald)$ in terms of the conditional probabilities of the graphical model.
- Compute and display in a human friendly format the conditional distributions $p(G=g \,|\, V=v)$ for all genders $g$ and votes $v$, by naively marginalising the other variables (other than $G$ and $V$, that is).

### Solution description

1. $$ p(R, G, H, J, A, C, V) = p(H)\ p(G)\ p(C\ |\ G, H)\ p(R)\ p(J\ |\ R, G)\ p(A\ |\ J)\ p(V\ |\ R,A,C)$$

2.

\begin{align*}
p(G \ | \ V) &= \frac{p(G,V)}{p(V)} \\
             &= \frac{1}{p(V)} \sum_{R,H,J,A,C} p(R, G, H, J, A,C, V) \\
             &= \frac{1}{p(V)} \sum_{R,H,J,A,C} p(H)\ p(G)\ p(C\ |\ G, H)\ p(R)\ p(J\ |\ R, G)\ p(A\ |\ J)\ p(V\ |\ R,A,C)
\end{align*}

So in particular, 

$$p(G = \text{male} \ | \ V = \text{Donald})) = \frac{p(G= \text{male})}{p(V= \text{Donald})} \sum_{R,H,J,A,C} p(H)\ p(C\ |\ G= \text{male}, H)\ p(R)\ p(J\ |\ R, G= \text{male})\ p(A\ |\ J)\ p(V= \text{Donald}\ |\ R,A,C)$$

Note that $p(V)$ has previously been written in terms of the conditional probabilities in the graphical model as well by similarly summing the factorisation of the joint pmf over the remaining variables.

In [17]:
def conditionals(network, node1, node2):
    '''
    Given a DAG and two nodes, this function returns a dictionary whose keys are all pairs 
    of possible outcomes of the two nodes, and value is p(node1 | node 2). It computes the
    values by naively marginalising over the variables in network other than node 1 and
    node 2 as done in the question above.
    '''
    
    # Form all possible outcome pairs for node 1 and node 2
    outcome_pairs = list(itertools.product(node1.outcomes, node2.outcomes))
    conditional_dict = dict() # Initialize dictionary that will be returned
    node2_marginals = marginals(network)[node2.name]
    
    for node1_outcome, node2_outcome in outcome_pairs:
            marginalized_nodes = dfs(network)
            marginalized_nodes.remove(node1)
            marginalized_nodes.remove(node2)
            
            node1_given_node2_cumsum = 0
            
            # Iterate over product of outcomes for marginalized variables, summing the pmf
            for marginalized_outcomes in itertools.product(*tuple(node.outcomes for node in marginalized_nodes)):
                #Form appropriate dictionary of outcomes for each tuple
                outcome_dictionary = dict(zip(marginalized_nodes, marginalized_outcomes))
                outcome_dictionary[node1] = node1_outcome
                outcome_dictionary[node2] = node2_outcome
                
                node1_given_node2_cumsum += joint_pmf(outcome_dictionary, network)
                
            conditional_dict[(node1_outcome),(node2_outcome)] = node1_given_node2_cumsum/node2_marginals[node2_outcome] 
    return conditional_dict
    
    

In [18]:
gender_given_vote = conditionals(election_model, gender, vote)

print('Gender  Vote     p(Gender|Vote)')
print('-------------------------------')
for (gender, vote), conditional in gender_given_vote.items():
    print(gender,' '*(6 - len(gender)), vote, ' '*(7-len(vote)), conditional)

Gender  Vote     p(Gender|Vote)
-------------------------------
male    bernie   0.398132990181
male    donald   0.214321628636
male    hillary  0.363555529929
male    ted      0.278437575417
female  bernie   0.601867009819
female  donald   0.785678371364
female  hillary  0.636444470071
female  ted      0.721562424583


### 1B6 Variable elimination

Denote the graphical model consider thus far $\mathcal{M}$.

1. Derive $p(R,G,J,A,C,V)$ by marginalising over $H$ in $\mathcal{M}$. 
- Describe how the structure (connectivity) of the new graphical model (call it $\mathcal{M}_H$) over all variables other than $H$ changes after eliminating $H$ in this way.
- Describe which conditional(s) in the new graphical model differ from the original model.
- Encode the $\mathcal{M}_H$ in python similarly to $\mathcal{M}$. 

1.

\begin{align*}
p(R, G, J, A, C, V) &= \sum_{H} p(R, G, H, J, A, C, V) \\
                    &= \sum_{H} p(H)\ p(G)\ p(C\ |\ G, H)\ p(R)\ p(J\ |\ R, G)\ p(A\ |\ J)\ p(V\ |\ R,A,C)\\
                    &= p(G)\ p(R)\ p(J\ |\ R, G)\ p(A\ |\ J)\ p(V\ |\ R,A,C) \sum_{H} p(H)\ p(C\ |\ G, H)
\end{align*}

By the Markov property at $G$ we can see that $p(G,H) = p(G)\ p(H),$ so 

$$ p(C\ | \ G,H) = \frac{ p(C,G,H) }{p(G,H)} = \frac{ p(C,G,H) }{p(G)\ p(H)}$$

So the last sum simplifies as: 

\begin{align*}
\sum_{H} p(H)\ p(C\ |\ G, H) &= \sum_{H} \frac{ p(C,G,H)}{p(G)} \\
                             &= \frac{ p(C,G) }{p(G)} \\
                             &= p(C \ | \ G)
\end{align*}

Therefore we have 

$$ p(R, G, J, A, C, V) = p(G)\ p(C\ | \ G)\ p(R)\ p(J\ |\ R, G)\ p(A\ |\ J)\ p(V\ |\ R,A,C)$$


 
2.

The new graphical model $\mathcal{M}_H$ is the same as $\mathcal{M}$ except the node $H$ and the arrow from $H$ to $C$ is removed.

3.

All remaining nodes in the new model are conditioned in the same way except for $C,$ which now is conditioned only on $G.$

4.

In [19]:
region_H = RandomVariable(
    name='region',
    parents=tuple(),
    outcomes=('north', 'south', 'east', 'west'), 
    pmf_array = region_pmf_array)


gender_H = RandomVariable(
    name='gender',
    parents=tuple(),
    outcomes=('male', 'female'), 
    pmf_array = gender_pmf_array)

jacket_H = RandomVariable(
    name='jacket',
    parents=(region_H, gender_H),
    outcomes=('full', 'part', 'never'), 
    pmf_array = jacket_pmf_array)

age_H = RandomVariable(
    name='age',
    parents=(jacket_H, ),
    outcomes=('new', 'worn', 'old'), 
    pmf_array = age_pmf_array)


colour_pmf_array_H = np.array(
        [
            [0.893,0.183],
            [0.107,0.817],
        ]
    )
colour_H = RandomVariable(
    name='colour',
    parents=(gender_H, ),
    outcomes=('black', 'white'),
    pmf_array = colour_pmf_array_H)

vote_H = RandomVariable(
    name='vote',
    parents=(region_H, age_H, colour_H),
    outcomes=vote_outcomes,
    pmf_array = vote_pmf_array)

election_model_H = BayesianNetwork(region_H, gender_H, jacket_H, age_H, colour_H, vote_H)

### 1B6 General estimation of conditional probabilities (revisited)

1. As you did earlier, compute and display in a human friendly format the conditional distributions $p(G=g \,|\, V=v)$ for all genders $g$ and votes $v$, by naively marginalising the other variables (other than $G$ and $V$, that is). This time however, do so using $\mathcal{M}_H$ rather than $\mathcal{M}$.
- Quantify the computational advantages of using $\mathcal{M}_H$ in this way rather than $\mathcal{M}$.
- Which variable (or variables) would be the best to eliminate in this way, in terms of the aforementioned computational advantages, and why?
- Pick a (best) variable from the previous question and call it $X$. Assuming we have eliminated $X$, which would be the "best" variable to subsequently eliminate in a similar fashion?

1.

In [20]:
gender_given_vote_H = conditionals(election_model_H, gender_H, vote_H)

print('Gender  Vote     p(Gender|Vote)')
print('-------------------------------')
for (gender, vote), conditional in gender_given_vote_H.items():
    print(gender,' '*(6 - len(gender)), vote, ' '*(7-len(vote)), conditional)

Gender  Vote     p(Gender|Vote)
-------------------------------
male    bernie   0.398132990181
male    donald   0.214321628636
male    hillary  0.363555529929
male    ted      0.278437575417
female  bernie   0.601867009819
female  donald   0.785678371364
female  hillary  0.636444470071
female  ted      0.721562424583


2.

We compute the conditional via the expression $$ p(G \ | V) = \frac{p(G, V)}{p(V)}$$

The numerator and denominator were computed by summing over the marginalized variables. To analyse the relative computation loads, we focus on counting the number of multiplications taken to reach the result as the additions are relatively small. We count this for computation of the denominator (the numerator takes half as many multiplications). Throughout the computation, keep in mind that $R, A, C, J, H, G$ have $4, 3, 2, 3, 2, 2$ possible values respectively. 

Computing as we did originally, 

$$ p(V=v) = \sum_{R,A,C,J,H,G} p(H)\ p(G)\ p(C\ |\ G, H)\ p(R)\ p(J\ |\ R, G)\ p(A\ |\ J)\ p(V=v\ |\ R,A,C)$$

This has $4 \cdot 3 \cdot 2 \cdot 3 \cdot 2 \cdot 2 = 288$ terms in the sum. Each term has $7$ factors (whose values are precomputed) so each term of the sum takes $6$ multiplications to evaluate.  So this computation requires $6\ \cdot \ 288 = 1728$ multiplications. 

Now we look at the same computation as done using $\mathcal{M}_H.$ This exhibits the process of "elimination of variables" for $H$, but the same steps can be applied for any variable other than $G,V,$ not just $H.$ 

The new computation was essentially to push the sum over $H$ and any terms that depend on $H$ inside and rewrite the marginal as follows:

\begin{align*}
p(V=v) &= \sum_{R,A,C,J,G} p(G)\ p(R)\ p(J\ |\ R, G)\ p(A\ |\ J)\ p(V=v\ |\ R,A,C) \sum_H p(H)\ p(C\ |\ G, H)\\
       &= \sum_{R,A,C,J,G} p(G)\ p(R)\ p(J\ |\ R, G)\ p(A\ |\ J)\ p(V=v\ |\ R,A,C) \ \cdot \  f_1 (C, G)
\end{align*}

where $f_1$ is a function of $C$ and $G$ only. We computed the values of $f_1$ for all possible $C,G$ and reused when it came time to compute the marginal. There are $2 \cdot 2 = 4$ pairs in the domain of $f_1.$ The computation for each is a sum of $2$ terms, each term requiring $1$ multiplication. So the computation of $f_1$ required $8$ multiplications. 

Now, to compute $p(V=v)$ taking advantage of our values for $f_1,$ we have a sum with $4\ \cdot \ 3\ \cdot \ 2\ \cdot \ 3\ \cdot \ 2 = 144$ terms, each term being a product of $6$ factors so requiring $5$ multiplications each. So this final part of the computation takes $144 \ \cdot \  5 = 720$ multiplications. Taking into account the multiplications required in computing $f_1,$ the total number of multiplications ia $728.$

So calculating conditions from the new model should are roughly $2.5$ times faster than the original model.

3 . 

Here are the results from repeating the same calculation for the other variables that we did for $H:$
\begin{align*}
R &: 576 \\
A &: 552 \\
C &: 816 \\
J &: 552
\end{align*}

So in terms of computation advantage, it is best to eliminate either $A$ or $J.$

4.

Suppose we choose to eliminate $A$ first. Then by calculations of the same style as above, we list the number of multiplications required to compute $p(V=v)$ if we eliminate another variable after $A.$

\begin{align*}
J &: 248 \\
R &: 244 \\
C &: 360 \\
H &: 272
\end{align*}

So if we picked $A$ to eliminate first and will eliminate one more, than the best one to pick next is $R.$